In [1]:
import datetime
import time
import os
import sys
import pytz
from pathlib import Path

import numpy as np
import pandas as pd

import astroplan
import skyfield

import astropy.units as u

from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from astropy.time import Time

from skyfield import api
from skyfield import almanac

import matplotlib
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

def nearest_minute(dt):
    # USNO predictions round to nearest minute so we use this hack to follow that
    return (dt + datetime.timedelta(seconds=30)).replace(second=0, microsecond=0)

ts = api.load.timescale()
e = api.load('de421.bsp')
tz = pytz.timezone("America/Phoenix")
lat = "31:41:19.6"
lon = "-110:53:04.4"
elevation = 2600 * u.m
location = EarthLocation.from_geodetic(lon, lat, elevation)
mmt = astroplan.Observer(name="MMTO", location=location, timezone="US/Arizona", pressure=0*u.mbar)
tset = mmt.sun_set_time(Time(datetime.datetime.now()), which='next', horizon=-0.8333*u.deg)
tset.isot
mmt_sf = api.Topos(latitude_degrees=location.lat.value, longitude_degrees=location.lon.value, elevation_m=2600.0)

In [3]:
test_coord = SkyCoord(20, 32, unit="deg")

In [25]:
%%prun -T blah.txt
for i in range(10):
    aa_frame = AltAz(obstime=Time.now(), location=location, temperature=5 * u.deg_C, pressure=775 * u.mbar)
    test_coord.transform_to(aa_frame)

 
*** Profile printout saved to text file 'blah.txt'.


         12276 function calls (11790 primitive calls) in 0.009 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  863/546    0.000    0.000    0.004    0.000 {built-in method builtins.getattr}
      136    0.000    0.000    0.001    0.000 quantity.py:409(__new__)
      134    0.000    0.000    0.000    0.000 baseframe.py:816(get_representation_component_names)
   152/96    0.000    0.000    0.001    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       13    0.000    0.000    0.001    0.000 formats.py:944(set_jds)
      236    0.000    0.000    0.000    0.000 decorators.py:769(__get__)
       48    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        2    0.000    0.000    0.002    0.001 iers.py:357(_interpolate)
      232    0.000    0.000    0.000    0.000 {built-in method numpy.array}
     1007    0.000    0.000    0.000    0.000 {built-in method builtins.isins

In [4]:
%%timeit
for i in range(100):
    aa_frame = AltAz(obstime=Time.now(), location=location, temperature=5 * u.deg_C, pressure=775 * u.mbar)
    test_coord.transform_to(aa_frame)

160 ms ± 723 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
